<a href="https://colab.research.google.com/github/lubianat/wikidata_markers/blob/master/reconcile_cell_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [22]:
worksheet = gc.open('Biocuration of Cell Classes for Wikidata' ).sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame.from_records(rows)

[['label', 'subclass of', 'stated in', 'described by source', 'aliases', 'anatomical location', 'develops from', 'instance of', '', '', '', '', ''], ['zebrafish  enteric neural crest cell ', 'Q106249115', 'Q106248504', '', '', '', '', '', '', '', '', '', ''], ['zebrafish early differentiating enteric neurons', 'Q101062626', 'Q106248504', '', '', '', '', '', '', '', '', '', ''], ['mouse epididymal cell', 'Q106517162', 'Q98166912', '', '', 'epididymis', '', '', '', '', '', '', ''], ['mouse epididymal  clear cell', 'Q106517168', 'Q98166912', 'Q98166912', '', 'epididymis', '', '', '', '', '', '', ''], ['mouse epididymal  basal cell', 'Q106517168', 'Q98166912', '', '', 'epididymis', '', '', '', '', '', '', ''], ['mouse epididymal  stromal cell', 'Q106517168', 'Q98166912', '', '', 'epididymis', '', '', '', '', '', '', ''], ['mouse epididymal  endothelial cell', 'Q106517168', 'Q98166912', '', '', 'epididymis', '', '', '', '', '', '', ''], ['mouse epididymal  immune cell', 'Q106517168', 'Q9816

In [23]:
# From https://stackoverflow.com/questions/31328861/python-pandas-replacing-header-with-top-row
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header


In [24]:
instance_dict = {
    "cell type":"Q189118",
    "cell class":"Q104852483",
    "cell state":"Q104539563"
}

SyntaxError: ignored

In [25]:
df

,label,subclass of,stated in,described by source,aliases,anatomical location,develops from,instance of,,,,,
1,zebrafish enteric neural crest cell,Q106249115,Q106248504,,,,,,,,,,
2,zebrafish early differentiating enteric neurons,Q101062626,Q106248504,,,,,,,,,,
3,mouse epididymal cell,Q106517162,Q98166912,,,epididymis,,,,,,,
4,mouse epididymal clear cell,Q106517168,Q98166912,Q98166912,,epididymis,,,,,,,
5,mouse epididymal basal cell,Q106517168,Q98166912,,,epididymis,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,human IgG plasmablast,Q106596039,Q106272047,,,,,,,,,,
192,nasal suprabasal cell,Q104807928,Q106272047,,suprabasal N,,,,,,,,
193,nasal multiciliated cell,Q101062671,Q106272047,,,,,,,,,,
194,human nasal suprabasal cell,nasal suprabasal cell,,,,,,,,,,,


In [30]:
import re
for index, row in df.iterrows():

    print("CREATE")

    label = row["label"]
    print(f'LAST|Len|"{label}"')

    stated_in = row["stated in"]
    if "human" in label:
      description = "Cell type in Homo sapiens"
      taxon = "Q15978631"
      print(f"LAST|P703|{taxon}|S248|{stated_in}")

    elif "mouse" in label:
      description = "Cell type in Mus musculus"
      taxon = "Q83310"
      print(f"LAST|P703|{taxon}|S248|{stated_in}")

    elif "zebrafish" in label:
      description = "Cell type in Danio rerio"
      taxon = "Q169444"
      print(f"LAST|P703|{taxon}|S248|{stated_in}")

    else:
      description = "Cell type"
    
    print(f'LAST|Den|"{description}"')
    
    if row["aliases"] != "":
        print(f'LAST|Aen|"{aliases}"')


    described_by_source = row["described by source"]

    if row["instance of"] != "":
      instance = instance_dict[row["instance of"]]
    else: 
      instance = instance_dict["cell type"]
    print(f"LAST|P31|{instance}|S248|{stated_in}")

    try:
        if  re.findall("Q[0-9]*", row["subclass of"]):
            subclass = row["subclass of"]
        else:
            subclass = subclass_dict[row["subclass of"]]            
        print(f"LAST|P279|{subclass}|S248|{stated_in}")

        try:
            part_of = part_of_dict[row["anatomical location"]]
            print(f"LAST|P927|{part_of}|S248|{stated_in}")
        except:
            pass





        if row["described by source"] != "":
            print(f'LAST|P1343|{described_by_source}')
        print("")
        
    except:
        print("missing: " + str(row["subclass of"]))
        break
    print(" ")

CREATE
LAST|Len|"zebrafish  enteric neural crest cell "
LAST|P703|Q169444|S248|Q106248504
LAST|Den|"Cell type in Danio rerio"
LAST|P31|Q189118|S248|Q106248504
LAST|P279|Q106249115|S248|Q106248504

 
CREATE
LAST|Len|"zebrafish early differentiating enteric neurons"
LAST|P703|Q169444|S248|Q106248504
LAST|Den|"Cell type in Danio rerio"
LAST|P31|Q189118|S248|Q106248504
LAST|P279|Q101062626|S248|Q106248504

 
CREATE
LAST|Len|"mouse epididymal cell"
LAST|P703|Q83310|S248|Q98166912
LAST|Den|"Cell type in Mus musculus"
LAST|P31|Q189118|S248|Q98166912
LAST|P279|Q106517162|S248|Q98166912
LAST|P927|Q1973610|S248|Q98166912

 
CREATE
LAST|Len|"mouse epididymal  clear cell"
LAST|P703|Q83310|S248|Q98166912
LAST|Den|"Cell type in Mus musculus"
LAST|P31|Q189118|S248|Q98166912
LAST|P279|Q106517168|S248|Q98166912
LAST|P927|Q1973610|S248|Q98166912
LAST|P1343|Q98166912

 
CREATE
LAST|Len|"mouse epididymal  basal cell"
LAST|P703|Q83310|S248|Q98166912
LAST|Den|"Cell type in Mus musculus"
LAST|P31|Q189118|S24